# **Project 4: LLM Project Activity - Topic Modeling**




### **Week 23** Project 4: 3-Pre-Trained-Model
* Select a pre-trained model for your project and perform data preprocessing. (9.4)
* Apply transfer learning concepts to enhance the model, followed by evaluating and optimizing the project model and creating an LLM Model Report. (9.5)

**Observation:**
After research, conclude that Hugging Face's pipeline does not have built-in support for topic modeling as it does for sentiment analysis or summarization. As topic modeling is an unsupervised NLP task, will require use of a different library. In response, will use a pretrained transformer model for embedding through BERTopic.
Notes: given this requirement, the course requests run this Hugging Face pipeline format:

Hugging Face pipeline()
data = ['list of the text for inference']
preds = pipe(data)

But as this process will use BERTopic, the following will be used to match the pipeline format and complete required passing of Text Data correctly:
docs = ds_test_df['text'].tolist()
topics, probs = topic_model.transform(docs)

Additionally, Hugging Face pipeline expects:
- A list of strings = returns predictions (classification, generation, etc.)

BERTopic expects the same input format:
- A list of strings (documents) = returns topics and their probabilities.

Select a pre-trained model for your project and perform data preprocessing. (9.4)

In [ ]:
#install required packages
!pip install bertopic

In [ ]:
#selected and loaded pre-trained BERTopic model from Hugging Face
from bertopic import BERTopic
topic_model = BERTopic.load("guibvieira/topic_modelling")

topic_model.get_topic_info()

In [ ]:
from datasets import load_dataset
!pip install -U fsspec datasets

In [ ]:
ds = load_dataset('SetFit/20_newsgroups')
import pandas as pd
ds_train = pd.DataFrame(ds['train'])
ds_test = pd.DataFrame(ds['test'])

In [ ]:
from datasets import Dataset, DatasetDict

train = Dataset.from_pandas(ds_train)
test = Dataset.from_pandas(ds_test)

new_ds = DatasetDict({
    'train': train,
    'test': test
})

In [ ]:
#pre-processing to improve embeddings and clustering quality in BERTopic
#text cleaning
import pandas as pd # Import pandas if not already imported
from datasets import load_dataset # Import load_dataset if not already imported
import re

# Load the dataset and convert to DataFrame if not already available
if 'ds_test' not in globals():
    ds = load_dataset('SetFit/20_newsgroups')
    ds_test = pd.DataFrame(ds['test'])

# Define 'docs' from the 'text' column of ds_test
docs = ds_test['text'].tolist()


def clean_text(text):
    text = text.lower() #lowercase
    text = re.sub(r"http\S+", "", text) #remove URLs
    text = re.sub(r"\n|\r", " ", text) #remove newlines
    text = re.sub(r"[^a-zA-Z\s]", "", text) #remove punctuation/symbols
    text = re.sub(r"\s+", " ", text).strip() #remove extra spaces
    return text

docs = [clean_text(doc) for doc in docs]

#pre-processing stowpword removal
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return " ".join([word for word in text.split() if word not in stop_words])

docs = [remove_stopwords(doc) for doc in docs]

#lemmatization/stemming (reduce words to root form)
import spacy
# Download the spacy model if not already downloaded
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    !python -m spacy download en_core_web_sm
    nlp = spacy.load("en_core_web_sm")


def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if token.lemma_ != "-PRON-"])

docs = [lemmatize_text(doc) for doc in docs]

Given this is a BERTopic model, transfer learning applies to the embedding model, not the topic model itself. The transfer learning will be using a pre-trained sentence transformer and fine-tuning the embedding model.

In [ ]:
#topic model requires embedding model specified explicitly, added code to address error
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

#load sentence transformer embedding model; note: automatically includes tokenizer 'sentence transformers' embedded in BERTopic handling internally
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

#load BERTopic with pretrained topic model AND embedding model
topic_model = BERTopic.load("guibvieira/topic_modelling", embedding_model=embedding_model)

In [ ]:
# Convert Hugging Face test set back to pandas DataFrame
ds_test_df = new_ds['test'].to_pandas()
docs = ds_test_df['text'].tolist()

# Run BERTopic
topics, probs = topic_model.transform(docs)

In [ ]:
# See an overview of the topics (topic IDs, counts, names)
topic_model.get_topic_info()

In [ ]:
#see all topic IDs (top 25) with top words in DF, sorted by count (descending)
topic_info = topic_model.get_topic_info()
top_topics = topic_info[topic_info['Topic'] != -1].head(25)  # Exclude outliers (-1)

# Print top 25 topics with keywords
for topic_id in top_topics['Topic']:
    print(f"\nTopic {topic_id}:")
    print(topic_model.get_topic(topic_id))

In [ ]:
#inspect predictions (topics per document)
import pandas as pd

results_df = pd.DataFrame({
    "document": docs,
    "topic": topics,
    "probability": probs
})
print(results_df.head())

Notes on predictions output:

A higher probability (closer to 1.0) means the model is more confident in the topic assignment.

A lower probability (closer to 0.0) means the document is less clearly matched to a topic.

If the topic were -1, it would mean the model considers the document an outlier that doesn't fit any topic well.


In [ ]:
#how many labels (topics) predicted; unique topics (excluding -1 which means 'no topic assigned')
unique_topics = set(topics)
n_topics = len([t for t in unique_topics if t != -1])
print(f"Number of topics found: {n_topics}")

In [ ]:
#summary of all topics
topic_model.get_topic_info()

Apply transfer learning concepts to enhance the model, followed by evaluating and optimizing the project model and creating an LLM Model Report. (9.5)

From Hugging Face transformer documentation task guide, natural language processing, selected the text classifciation to apply as it may be the best fit for transfer learning in topic modeling.

In [ ]:
#text classfication - load necessary libraries from Hugging face
!pip install transformers datasets evaluate accelerate

In [ ]:
#load data set
from datasets import load_dataset
ds = load_dataset('SetFit/20_newsgroups')

In [ ]:
print(ds["test"][0])

In [ ]:
#preprocess load DistilBERT tokenizer to prepcess text field
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [ ]:
#create preprocessing function to tokenize text and truncate sequences to no longer than DistilBERT's max input length
# Define preprocessing function
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

# Tokenize all splits
tokenized_ds = ds.map(preprocess_function, batched=True)

In [ ]:
train_ds = tokenized_ds["train"]
eval_ds = tokenized_ds["test"]

In [ ]:
#create batch examples with DataCollatorWithPadding
#Pytorch
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")


In [ ]:
#Evaluate model performance
import evaluate
accuracy = evaluate.load("accuracy")

In [ ]:
#function to pass predictions and labels to compute calculate accuracy
import numpy as np
from evaluate import load  #Hugging Face's metric loading

accuracy = load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
#check dataset labels
print(set(ds["train"]["label"]))

In [ ]:
#before training, map expected ids to labels
#manually define label names
label_names = [
    'alt.atheism',
    'comp.graphics',
    'comp.os.ms-windows.misc',
    'comp.sys.ibm.pc.hardware',
    'comp.sys.mac.hardware',
    'comp.windows.x',
    'misc.forsale',
    'rec.autos',
    'rec.motorcycles',
    'rec.sport.baseball',
    'rec.sport.hockey',
    'sci.crypt',
    'sci.electronics',
    'sci.med',
    'sci.space',
    'soc.religion.christian',
    'talk.politics.guns',
    'talk.politics.mideast',
    'talk.politics.misc',
    'talk.religion.misc'
]

# Create mappings
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {label: i for i, label in enumerate(label_names)}

In [ ]:
#train model, load DistilBert
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased",
    num_labels=len(label_names),
    id2label=id2label,
    label2id=label2id
)

In [ ]:
#define training hyperparameters in TrainingArguments, choose appropriate metric for model to calculate metrics during training, pass training arguments to Trainer, call train() to finetune model;F1 will determine best model at end
#metrics selected: accuracy overall percentage of correctly classified documents and Macro F1 Score to determine mean of precision/recall per class and averaged
import numpy as np
from evaluate import load

# Load all required metrics
accuracy = load("accuracy")
f1 = load("f1")
precision = load("precision")
recall = load("recall")
roc_auc = load("roc_auc")  # AUC for multi-class classification

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "precision": precision.compute(predictions=preds, references=labels, average="macro")["precision"],
        "recall": recall.compute(predictions=preds, references=labels, average="macro")["recall"],
        "roc_auc_ovr": roc_auc.compute(prediction_scores=logits, references=labels, average="macro", multi_class="ovr")["roc_auc"]
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="newsgroups_model",             #checkpoints & model are saved
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    push_to_hub=False,
    save_strategy="steps",             # save checkpoints every N steps
    save_steps=25,
    save_total_limit=3,                # keep last 3 checkpoints to save disk space
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
import nbformat

input_path = "Project_4_3_pre-trained_model.ipynb"
output_path = "Project_4_3_pre-trained_model_cleaned.ipynb"

# Load the notebook
with open(input_path, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)

# Clear notebook-level metadata
nb.metadata = {}

# Clean each cell
for cell in nb.cells:
    cell.outputs = []
    cell.execution_count = None
    cell.metadata = {}

# Save cleaned notebook
with open(output_path, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

print(f"Cleaned notebook saved to {output_path}")

In [ ]:
import os
os.listdir()